In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
sys.path.append('..')
import theanomics.neuralnetwork as nn
from theanomics import learn, utils
from models import standard_model, all_conv_model, deepbind_model, residual_deepbind_model

#sys.path.append('')
import load_data

np.random.seed(247) # for reproducibility

In [2]:
# load data
data_path = '../../data/GenomeSimulation'
file_path = os.path.join(data_path, 'Localized_N=100000_S=200_M=50_G=20_data.pickle')
train, valid, test = load_data.simulation_pickle(file_path)

loading data from: /Users/juliankimura/Desktop/data/GenomeSimulation/Localized_N=100000_S=200_M=50_G=20_data.pickle
loading train data
loading cross-validation data
loading test data


In [3]:
# build network
input_shape = list(train[0].shape)
input_shape[0] = None
output_shape = train[1].shape
network, placeholders, optimization = residual_deepbind_model.model(input_shape, output_shape)

In [4]:
# build neural network class
nnmodel = nn.NeuralNet(network, placeholders)
nnmodel.inspect_layers()

# set output file paths
output_name = 'test'
utils.make_directory(data_path, 'Results')
file_path = os.path.join(data_path, 'Results', output_name)
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', file_path=file_path)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: 
shape:(None, 4, 200, 1)
layer2: 
shape:(None, 20, 200, 1)
parameters: W
layer3: 
shape:(None, 20, 200, 1)
parameters: beta, gamma, mean, inv_std
layer4: 
shape:(None, 20, 200, 1)
layer5: 
shape:(None, 20, 200, 1)
parameters: W
layer6: 
shape:(None, 20, 200, 1)
parameters: beta, gamma, mean, inv_std
layer7: 
shape:(None, 20, 200, 1)
layer8: 
shape:(None, 20, 200, 1)
parameters: W
layer9: 
shape:(None, 20, 200, 1)
parameters: beta, gamma, mean, inv_std
layer10: 
shape:(None, 20, 200, 1)
layer11: 
shape:(None, 20, 200, 1)
layer12: 
shape:(None, 20, 200, 1)
layer13: 
shape:(None, 100)
parameters: W
layer14: 
shape:(None, 100)
parameters: beta, gamma, mean, inv_std
layer15: 
shape:(None, 100)
layer16: 
shape:(None, 100)
layer17: 
shape:(None, 100)
parameters: W
layer18: 
shape:(None, 100)
parameters: beta, ga

In [5]:
# train model
learn.train_minibatch(nntrainer, data={'train': train, 'valid': valid}, 
                              batch_size=100, num_epochs=500, patience=10, verbose=1)

Epoch 1 out of 500 
[==                            ] 5.0% -- time=107s -- loss=0.45269 -- accuracy=80.10%  

KeyboardInterrupt: 

In [ ]:
# load best model --> lowest cross-validation error
nntrainer.set_best_parameters()

# test model
nntrainer.test_model(test, name="test", batch_size=100)

In [ ]:
# save all performance metrics (train, valid, test)
nntrainer.save_all_metrics(file_path)